In [2]:
import os
import operator
from num2words import num2words  # 这个包不支持中文
import gc
import pandas as pd
import numpy as np
import time

In [4]:
train_df = pd.read_csv('../input/en_train.csv')
test_df = pd.read_csv('../input/en_test_2.csv')
sample_df = pd.read_csv('../input/en_sample_submission.csv')

In [3]:
train_df.head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,Brillantaisia,Brillantaisia
1,0,1,PLAIN,is,is
2,0,2,PLAIN,a,a
3,0,3,PLAIN,genus,genus
4,0,4,PLAIN,of,of


In [14]:
# 根据训练数据，生成一个大辞典：res，并保留出现次数。
res = {}
def update_res_by_one_row(row):
#     print(type(row), row)
    # row[0] = before, row[1] = after
    if row[0] not in res:
        res[row[0]] = dict()
        res[row[0]][row[1]] = 1
    else:
        if row[1] in res[row[0]]:
            res[row[0]][row[1]] += 1
        else:
            res[row[0]][row[1]] = 1
# train_df.apply(update_res_by_one_row, axis=1)
import time
s = time.time()
for row in train_df[["before", "after"]].values:
    update_res_by_one_row(row)
print("cost time: "time.time() - s)

9.810082912445068


In [15]:
res

{nan: {nan: 46, 'n a': 5},
 'October 22, 1973': {'october twenty second nineteen seventy three': 1},
 'Wren': {'Wren': 21},
 'hypocorism': {'hypocorism': 2},
 'Comi': {'Comi': 2},
 'souanke': {'souanke': 1},
 "Watelet's": {"Watelet's": 1},
 '17 October 2013': {'the seventeenth of october twenty thirteen': 8},
 'twig': {'twig': 5},
 'Gramelspacher': {'Gramelspacher': 1},
 '$25,583': {'twenty five thousand five hundred eighty three dollars': 1},
 'WFAN': {'w f a n': 1},
 "Metuchen's": {"Metuchen's": 1},
 'Friday, November 08, 1850': {'friday november eighth eighteen fifty': 1},
 'Banija': {'Banija': 1},
 'Ferland': {'Ferland': 4},
 'Dalmore': {'Dalmore': 2},
 'Perspec': {'Perspec': 1},
 'Aurness': {'Aurness': 1},
 'Kellison': {'Kellison': 1},
 'Flandern': {'Flandern': 1},
 'ClubmanThe': {'ClubmanThe': 1},
 'uneven': {'uneven': 27},
 '26 March 1571': {'the twenty sixth of march fifteen seventy one': 1},
 'Lietuviskoji': {'Lietuviskoji': 1},
 "Meg's": {"Meg's": 2},
 "Rica's": {"Rica's": 2}

In [ ]:
total = 0
changes = 0
line = test.readline().strip()
while 1:
    line = test.readline().strip()
    if line == '':
        break

    pos = line.find(',')
    i1 = line[:pos]
    line = line[pos + 1:]

    pos = line.find(',')
    i2 = line[:pos]
    line = line[pos + 1:]

    line = line[1:-1]
    out.write('"' + i1 + '_' + i2 + '",')
    if line in res:
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        out.write('"' + srtd[0][0] + '"')
        changes += 1
    else:
        out.write('"' + line + '"')

    out.write('\n')
    total += 1

print('Total: {} Changed: {}'.format(total, changes))
test.close()
out.close()

In [5]:
# -*- coding: utf-8 -*-

import operator
from num2words import num2words
import os

INPUT_PATH = "input"
SUBM_PATH = "."

SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")
OTH = str.maketrans("፬", "4")

def train():
    print('Train start...')
    
    # Work with primary dataset
    file = "en_train.csv"
    train = open(os.path.join(INPUT_PATH, file), encoding='UTF8')
    line = train.readline()
    res = dict()
    total = 0
    not_same = 0
    while 1:
        line = train.readline().strip()
        if line == '':
            break
        total += 1
        pos = line.find('","')
        text = line[pos + 2:]
        if text[:3] == '","':
            continue
        text = text[1:-1]
        arr = text.split('","')
        if arr[0] != arr[1]:
            not_same += 1
        if arr[0] not in res:
            res[arr[0]] = dict()
            res[arr[0]][arr[1]] = 1
        else:
            if arr[1] in res[arr[0]]:
                res[arr[0]][arr[1]] += 1
            else:
                res[arr[0]][arr[1]] = 1
    train.close()
    print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))

    # Work with additional dataset from https://www.kaggle.com/google-nlu/text-normalization
    files = ['output_1.csv', 'output_6.csv', 'output_11.csv', 'output_16.csv', \
        'output_21.csv', 'output_91.csv', 'output_96.csv']

    for file in files:
        train = open(os.path.join(INPUT_PATH, 'tn', file), encoding='UTF8')
        line = train.readline()
        while 1:
            line = train.readline().strip()
            if line == '':
                break
            line = line.replace(',NA,', ',"NA",')
            total += 1
            pos = line.find('","')
            text = line[pos + 2:]
            if text[:3] == '","':
                continue
            text = text[1:-1]
            arr = text.split('","')
            if arr[0] == '<eos>':
                continue
            if arr[1] != '<self>':
                not_same += 1

            if arr[1] == '<self>' or arr[1] == 'sil':
                arr[1] = arr[0]

            if arr[0] not in res:
                res[arr[0]] = dict()
                res[arr[0]][arr[1]] = 1
            else:
                if arr[1] in res[arr[0]]:
                    res[arr[0]][arr[1]] += 1
                else:
                    res[arr[0]][arr[1]] = 1
        train.close()
        print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))
    
    return res

In [ ]:
def solve(res):
    sdict = {}
    sdict['km2'] = 'square kilometers'
    sdict['km'] = 'kilometers'
    sdict['kg'] = 'kilograms'
    sdict['lb'] = 'pounds'
    sdict['dr'] = 'doctor'
    sdict['m²'] = 'square meters'

    total = 0
    changes = 0
    out = open(os.path.join(SUBM_PATH, 'baseline2_en.csv'), "w", encoding='UTF8')
    out.write('"id","after"\n')
    test = open(os.path.join(INPUT_PATH, "en_test.csv"), encoding='UTF8')
    line = test.readline().strip()
    while 1:
        line = test.readline().strip()
        if line == '':
            break

        pos = line.find(',')
        i1 = line[:pos]
        line = line[pos + 1:]

        pos = line.find(',')
        i2 = line[:pos]
        line = line[pos + 1:]

        line = line[1:-1]
        out.write('"' + i1 + '_' + i2 + '",')
        if line in res:
            srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
            out.write('"' + srtd[0][0] + '"')
            changes += 1
        else:
            # line.split(' ')
            if len(line) > 1:
                val = line.split(',')
                if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                    line = ''.join(val)

            if line.isdigit():
                srtd = line.translate(SUB)
                srtd = srtd.translate(SUP)
                srtd = srtd.translate(OTH)
                out.write('"' + num2words(float(srtd)) + '"')
                changes += 1
            elif len(line.split(' ')) > 1:
                val = line.split(' ')
                for i, v in enumerate(val): 
                    if v.isdigit():
                        srtd = v.translate(SUB)
                        srtd = srtd.translate(SUP)
                        srtd = srtd.translate(OTH)
                        val[i] = num2words(float(srtd))
                    elif v in sdict:
                        val[i] = sdict[v]

                out.write('"' + ' '.join(val) + '"')
                changes += 1
            else:
                out.write('"' + line + '"')

        out.write('\n')
        total += 1

    print('Total: {} Changed: {}'.format(total, changes))
    test.close()
    out.close()

In [21]:
res = train()

Train start...
en_train.csv:	Total: 9918441 Have diff value: 659793
output_1.csv:	Total: 67101629 Have diff value: 12723310
output_6.csv:	Total: 124336086 Have diff value: 24795377
output_11.csv:	Total: 181494302 Have diff value: 36848944
output_16.csv:	Total: 238523401 Have diff value: 48886815
output_21.csv:	Total: 295674425 Have diff value: 60943615
output_91.csv:	Total: 352830815 Have diff value: 72996182
output_96.csv:	Total: 410013747 Have diff value: 85056247


In [ ]:
def solve2(res):
    sdict = {}
    sdict['km2'] = 'square kilometers'
    sdict['km'] = 'kilometers'
    sdict['kg'] = 'kilograms'
    sdict['lb'] = 'pounds'
    sdict['dr'] = 'doctor'
    sdict['m²'] = 'square meters'

    total = 0
    changes = 0
    out = open(os.path.join(SUBM_PATH, 'baseline_3_en.csv'), "w", encoding='UTF8')
    out.write('"id","after"\n')
    test = open(os.path.join(INPUT_PATH, "en_test.csv"), encoding='UTF8')
    line = test.readline().strip()
    while 1:
        line = test.readline().strip()
        if line == '':
            break

        pos = line.find(',')
        i1 = line[:pos]
        line = line[pos + 1:]

        pos = line.find(',')
        i2 = line[:pos]
        line = line[pos + 1:]

        line = line[1:-1]
        out.write('"' + i1 + '_' + i2 + '",')
        if line in res:
            srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
            out.write('"' + srtd[0][0] + '"')
            changes += 1
        else:
            if len(line) > 1:
                val = line.split(',')
                if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                    line = ''.join(val)

            if line.isdigit():
                srtd = line.translate(SUB)
                srtd = srtd.translate(SUP)
                srtd = srtd.translate(OTH)
                out.write('"' + num2words(float(srtd)) + '"')
                changes += 1
            elif len(line.split(' ')) > 1:
                val = line.split(' ')
                for i, v in enumerate(val):
                    if v.isdigit():
                        srtd = v.translate(SUB)
                        srtd = srtd.translate(SUP)
                        srtd = srtd.translate(OTH)
                        val[i] = num2words(float(srtd))
                    elif v in sdict:
                        val[i] = sdict[v]

                out.write('"' + ' '.join(val) + '"')
                changes += 1
            else:
                out.write('"' + line + '"')

        out.write('\n')
        total += 1

    print('Total: {} Changed: {}'.format(total, changes))
    test.close()
    out.close()

In [ ]:
solve2(res)